# Load an irregular time series pre-trained model (TSiT) and perform inference on a test dataset

## Libraries

In [1]:
from tsai.all import *
my_setup()
import os
import pickle

os              : Linux-5.10.218-llgrid-x86_64-with-glibc2.17
python          : 3.8.15
tsai            : 0.3.7
fastai          : 2.7.12
fastcore        : 1.5.29
torch           : 1.12.1+cu113
cpu cores       : 48
threads per cpu : 2
RAM             : 187.54 GB
GPU memory      : N/A


## Select folder

In [2]:
foldername = "pre_trained_model_Nst_10"
# foldername = "pre_trained_model_Nst_15"

export_file = 'export.pkl' 

## Load pre-trained model

In [3]:
learn_loaded = load_learner(fname=os.path.join(foldername, export_file)) 

class_p = ['PAYLOAD']
class_r = ['ROCKET BODY']
class_d = ['DEBRIS']
vocab = [class_d[0], class_p[0], class_r[0]]

## Load original dataset

In [4]:
file_path = os.path.join(foldername, 'split_data.pkl')
with open(file_path, 'rb') as f:
    X, y, splits, ID_pred, feat_names = pickle.load(f)
X_test_2 = X[splits[1]]
y_test_2 = [y[i] for i in splits[1]]

## Select one sample in the dataset (or add your sample)

In [5]:
item0 = 0
item1 = item0 + 1
X_test_2_b = X_test_2[item0:item1]
y_test_2_b = y_test_2[item0:item1]

## Perform inference on a test dataset

In [6]:
test_ds = []
test_dl = []

# Create a test dataset
test_ds = TSDatasets(X_test_2_b, y_test_2_b, tfms=[None, TSClassification(vocab=vocab)], splits=None) 
 # Create a test dataloader
test_dl = learn_loaded.dls.new(test_ds)

targets_pred, targets_true = learn_loaded.get_preds(dl=test_dl)
targets_pred = targets_pred.argmax(dim=-1)

loss = learn_loaded.top_losses(X_test_2_b, y_test_2_b, largest=True, k = len(y_test_2_b))
nn = loss[1]

aa = []
for i1 in range(len(nn)):
    aa.append([int(ID_pred[nn[i1]][0]), vocab[targets_true[nn[i1]].item()], vocab[targets_pred[nn[i1]].item()]])
    
# Create an empty dictionary
count_dict = {}

# Loop through the list
for _, true, predicted in aa:
    # If the true label is not in the dictionary, add it
    if true not in count_dict:
        count_dict[true] = {predicted: 1}
    else:
        # If the predicted label is not associated with the true label, add it
        if predicted not in count_dict[true]:
            count_dict[true][predicted] = 1
        else:
            # If the predicted label is already associated with the true label, increment the count
            count_dict[true][predicted] += 1

# # Print the dictionary
# for true, predicted_dict in count_dict.items():
#     print(f"For true label '{true}':")
#     for predicted, count in predicted_dict.items():
#         print(f"    Predicted label '{predicted}': {count} times")


# Print the dictionary
for true, predicted_dict in count_dict.items():
    print(f"True label: '{true}':")
    for predicted, count in predicted_dict.items():
        print(f"Predicted label: '{predicted}'")